In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, AvgPool2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Machine_Learning_SWE_427/ML_Lab_Final_Assignment/dataset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [5]:
import cv2
import glob

abdomen = []
chest = []
cxr = []
hand = []
head = []

for i in glob.glob("/tmp/CNN_dataset/AbdomenCT/*.jpeg"):
    abdomen.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/ChestCT/*.jpeg"):
    chest.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/CXR/*.jpeg"):
    cxr.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/Hand/*.jpeg"):
    hand.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/HeadCT/*.jpeg"):
    head.append(cv2.imread(i))


In [6]:
x,y = [],[]
dataset_size = len(hand)
for i in range(dataset_size):
  x.append(abdomen[i])
  y.append([1])

  x.append(chest[i])
  y.append([2])

  x.append(cxr[i])
  y.append([3])

  x.append(hand[i])
  y.append([4])

  x.append(head[i])
  y.append([5])


In [7]:
hand[0][55][0]

array([119, 119, 119], dtype=uint8)

In [8]:
x,y = np.array(x), np.array(y)

In [9]:
#defining training and test sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.33)

#Dimension of the dataset
print((x_train.shape,y_train.shape))
print((x_test.shape,y_test.shape))

((33500, 64, 64, 3), (33500, 1))
((16500, 64, 64, 3), (16500, 1))


In [10]:
#randomly distributed
y_train[:10]

array([[1],
       [5],
       [5],
       [5],
       [1],
       [4],
       [2],
       [1],
       [3],
       [5]])

In [11]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [12]:
model = tf.keras.Sequential()
model.add(Conv2D(filters=32, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu', input_shape=(64,64,3))) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.85))
model.add(Conv2D(filters=16, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 16)        2064      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 16)        0

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
x_train = x_train.reshape(-1, 64, 64,3) #Reshape for CNN 
x_test = x_test.reshape(-1, 64, 64, 3)
x_train.shape

(33500, 64, 64, 3)

In [15]:
model_log=model.fit(x_train, y_train,
          batch_size=80,
          epochs=5,
          verbose=1,
          validation_split=.3)

Epoch 1/5
294/294 [==============================] - 16s 16ms/step - loss: 0.4074 - accuracy: 0.8597 - val_loss: 0.3395 - val_accuracy: 0.9241
Epoch 2/5
294/294 [==============================] - 3s 12ms/step - loss: 0.1070 - accuracy: 0.9739 - val_loss: 0.1488 - val_accuracy: 0.9358
Epoch 3/5
294/294 [==============================] - 4s 13ms/step - loss: 0.0495 - accuracy: 0.9874 - val_loss: 0.0488 - val_accuracy: 0.9914
Epoch 4/5
294/294 [==============================] - 4s 14ms/step - loss: 0.0391 - accuracy: 0.9898 - val_loss: 0.0222 - val_accuracy: 0.9977
Epoch 5/5
294/294 [==============================] - 4s 12ms/step - loss: 0.0276 - accuracy: 0.9919 - val_loss: 0.0191 - val_accuracy: 0.9971


In [19]:
predict_y = model.predict(x_test)
y_pred=np.argmax(predict_y,axis=1)

516/516 [==============================] - 2s 3ms/step


In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9972727272727273